In [4]:
import json

# 파일 경로 설정 (네 로컬 경로에 맞게 수정)
with open("laws_detailed.json", encoding="utf-8") as f:
    laws_data = json.load(f)

with open("deceptive_design_cases.json", encoding="utf-8") as f:
    cases_data = json.load(f)

with open("dark_patterns.json", encoding="utf-8") as f:
    pattern_data = json.load(f)

with open("all_pattern_predicates.json", encoding="utf-8") as f:
    predicate_data = json.load(f)

# 1. predicate → pattern
predicate_type_facts = []
for pattern, predicates in predicate_data.items():
    for predicate in predicates:
        pred_name = predicate.split("(")[0]  # 인자 제거
        fact = f"predicate_type({pred_name}, {pattern.lower().replace(' ', '_')})."
        predicate_type_facts.append(fact)

# 2. pattern → law
type_law_facts = []
for law in laws_data:
    law_title = law["title"].lower().replace(" ", "_").replace(",", "").replace(".", "").replace("§", "s")
    for pattern in law.get("related_deceptive_patterns", []):
        pattern_name = pattern["name"].lower().replace(" ", "_")
        fact = f"type_law({pattern_name}, {law_title})."
        type_law_facts.append(fact)

# 3. case → predicate (유추 기반)
case_predicate_facts = []
for case_name, case_info in cases_data.items():
    case_id = case_name.lower().replace(" ", "_").replace(",", "").replace("'", "").replace("’", "")
    for pattern_desc in case_info.get("Related deceptive patterns", []):
        pattern_key = pattern_desc.split(" ")[0].lower()
        for predicate in predicate_data.get(pattern_key, []):
            pred_name = predicate.split("(")[0]
            fact = f"case_predicate({case_id}, {pred_name})."
            case_predicate_facts.append(fact)

# 모든 fact 통합
facts_lines = (
    predicate_type_facts +
    [""] +
    type_law_facts +
    [""] +
    case_predicate_facts
)

# 저장
with open("facts.pl", "w", encoding="utf-8") as f:
    f.write("\n".join(facts_lines))

print("facts.pl 생성 완료!")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)